In [1]:
import pandas as pd

In [58]:
df_DRE = pd.read_csv('historic_data/historico_DRE.csv',sep=';',encoding='latin-1')

In [59]:
df_DRE.columns = [k.strip() for k in df_DRE.columns]
df_DRE.columns

Index(['safra', 'custo de bens e ou serviços vendidos',
       'deduções da receita bruta', 'despesas', 'despesas com vendas',
       'despesas da intermediação financeira', 'despesas de pessoal',
       'despesas financeiras', 'despesas gerais e administrativas',
       'despesas tributárias', 'financeiras', 'ir diferido',
       'lucro prejuízo do período', 'outras despesas administrativas',
       'outras despesas operacionais', 'outras despesas receitas operacionais',
       'outras receitas operacionais', 'part. de acionistas não controladores',
       'participações contribuições estatutárias',
       'perdas pela não recuperabilidade de ativos',
       'provisão para ir e contribuição social',
       'receita bruta de vendas e ou serviços',
       'receita líquida de vendas e ou serviços', 'receitas',
       'receitas da intermediação financeira',
       'receitas de prestação de serviços', 'receitas financeiras',
       'resultado antes tributação participações', 'resultado bru

In [15]:
# (3) = (1)-(2)
# (5) = (3)-(4)
# (8) = (5)-[(6)-(7.1)+(7.2)]
# (10) = (8)-(9)
# (12) = (10)-[(11.2)-(11.1)]
# (15) = (12)-(13.1)+(13.2)-(14)

# Para financeiras, receitas e despesas da intermediação financeira são receita bruta e deduções sobre ela
# "outras despesas receitas operacionais" - somar direto com valor original (pode mesclar positivo ou negativo)

dict_cols = {'1 - Receita bruta':['receita bruta de vendas e ou serviços','receitas da intermediação financeira'],
             '2 - Deduções sobre receita bruta':['deduções da receita bruta'],
             '3 - Receita líquida':['receita líquida de vendas e ou serviços'],
             '4 - Custo de mercadorias e serviços vendidos (CMV)':['custo de bens e ou serviços vendidos',
                                                                   'despesas da intermediação financeira'],
             '5 - Resultado bruto':['resultado bruto','resultado bruto intermediação financeira'],
             '6 - Despesas com vendas':['despesas com vendas'],
             '7.1 - Receitas operacionais':['outras receitas operacionais','receitas',
                                            'outras despesas receitas operacionais',
                                            'receitas de prestação de serviços'],
             '7.2 - Despesas gerais, administrativas e de pessoal':['despesas gerais e administrativas',
                                                                    'perdas pela não recuperabilidade de ativos',
                                                                    'despesas', 'despesas de pessoal',
                                                                    'despesas tributárias','outras despesas operacionais',
                                                                    'outras despesas administrativas'],
             '8 - EBITDA':[],
             '9 - Depreciações e amortizações':[],
             '10 - EBIT':[],
             '11.1 - Receitas financeiras':['receitas financeiras'],
             '11.2 - Despesas financeiras':['despesas financeiras'],
             '12 - EBT':['resultado antes tributação participações'],
             '13.1 - IR e taxas':['provisão para ir e contribuição social'],
             '13.2 - Restituição de IR e taxas':['ir diferido'],
             '14 - Participação de acionistas':['part. de acionistas não controladores',
                                                'participações contribuições estatutárias',
                                                'reversão dos juros sobre capital próprio'],
             '15 - Resultado do período':['lucro prejuízo do período']
            }
vars_consolidacao_originais_base = ['resultado bruto','resultado da equivalência patrimonial','financeiras',
                                    'resultado não operacional','resultado antes tributação participações',
                                    'lucro prejuízo do período','resultado bruto intermediação financeira',
                                    'resultado operacional']

In [16]:
cols_consideradas = []
for col in list(dict_cols.values()):
    cols_consideradas = cols_consideradas + col
cols_n_consideradas = ((set(df_DRE.columns) - set(['safra','ticker'])) - set(cols_consideradas)) - set(vars_consolidacao_originais_base)

In [66]:
df_DRE[df_DRE['ticker']=='SQIA3']['provisão para ir e contribuição social']

18003    -558.000
18004    -216.000
18005   -2216.000
18006   -1246.000
18007    -281.000
18008    -206.000
18009    -650.000
18010    -264.000
18011    -101.000
18012     -51.000
18013   -1200.000
18014    -275.000
18015    -923.000
18016    -862.000
18017     929.312
18018    -186.478
18019     362.494
18020    -591.810
18021    -525.328
18022   -1544.515
18023     649.372
18024    -387.851
18025      23.723
18026    -292.158
18027    -255.940
18028     -35.378
18029   -1446.546
18030    -313.468
18031    -399.016
18032    -219.645
Name: provisão para ir e contribuição social, dtype: float64

In [112]:
def concatena_cols_df(df,cols):
    df = df.copy()
    df['dummy_col'] = 0
    for col in cols:
        df['dummy_col'] = df['dummy_col'] + df[col].fillna(0)
    return df['dummy_col']

df_DRE_simp = df_DRE[['safra','ticker']].copy()
for lin in dict_cols.keys():
    if '3 - ' in lin:
        df_DRE_simp[lin] = concatena_cols_df(df_DRE,dict_cols[lin])
        df_DRE_simp[lin] = [l1 - l2 if l3==0 else l3 for (l1,l2,l3) in zip(df_DRE_simp['1 - Receita bruta'],\
                                                                           df_DRE_simp['2 - Deduções sobre receita bruta'],\
                                                                           df_DRE_simp['3 - Receita líquida'])]
    elif '5 - ' in lin and 'bruto' in lin:
        df_DRE_simp[lin] = df_DRE_simp['3 - Receita líquida'] + \
        df_DRE_simp['4 - Custo de mercadorias e serviços vendidos (CMV)']
    elif '8 - ' in lin:
        df_DRE_simp[lin] = df_DRE_simp['5 - Resultado bruto'] + df_DRE_simp['6 - Despesas com vendas'] \
        + df_DRE_simp['7.1 - Receitas operacionais'] + df_DRE_simp['7.2 - Despesas gerais, administrativas e de pessoal']
    elif '9 - ' in lin:
        df_DRE_simp[lin] = 0
    elif '10 - ' in lin:
        df_DRE_simp[lin] = df_DRE_simp['8 - EBITDA'] + df_DRE_simp['9 - Depreciações e amortizações']
    elif '12 - ' in lin:
        df_DRE_simp[lin] = df_DRE_simp['10 - EBIT'] + df_DRE_simp['11.2 - Despesas financeiras'] \
        + df_DRE_simp['11.1 - Receitas financeiras']
    elif '15 - ' in lin:
        df_DRE_simp[lin] = df_DRE_simp['12 - EBT'] + df_DRE_simp['13.1 - IR e taxas'] + \
        df_DRE_simp['13.2 - Restituição de IR e taxas'] + df_DRE_simp['14 - Participação de acionistas']
    else:
        df_DRE_simp[lin] = concatena_cols_df(df_DRE,dict_cols[lin])

In [114]:
df_DRE_simp[df_DRE_simp['ticker']=='ITUB3']

,safra,ticker,1 - Receita bruta,2 - Deduções sobre receita bruta,3 - Receita líquida,4 - Custo de mercadorias e serviços vendidos (CMV),5 - Resultado bruto,6 - Despesas com vendas,7.1 - Receitas operacionais,"7.2 - Despesas gerais, administrativas e de pessoal",8 - EBITDA,9 - Depreciações e amortizações,10 - EBIT,11.1 - Receitas financeiras,11.2 - Despesas financeiras,12 - EBT,13.1 - IR e taxas,13.2 - Restituição de IR e taxas,14 - Participação de acionistas,15 - Resultado do período
11184,30/06/2019,ITUB3,8.834110e+05,0.0,8.834110e+05,-3.595050e+05,523906.016,0.0,6086989.152,-159114.000,6451781.168,0,6451781.168,0.0,0.0,6451781.168,402220.000,0.000,-5970.000,6848031.168
11185,31/03/2019,ITUB3,1.505153e+06,0.0,1.505153e+06,-7.345010e+05,770652.032,0.0,5837411.840,-201105.000,6406958.872,0,6406958.872,0.0,0.0,6406958.872,-115347.000,0.000,-7211.000,6284400.872
11186,31/12/2018,ITUB3,3.982182e+05,0.0,3.982182e+05,3.297930e+05,728011.264,0.0,6957605.320,-86086.000,7599530.584,0,7599530.584,0.0,0.0,7599530.584,-1319264.000,278023.008,-4417.000,6553872.592
11187,30/09/2018,ITUB3,2.447113e+06,0.0,2.447113e+06,-1.741973e+06,705139.968,0.0,4533392.920,-176325.000,5062207.888,0,5062207.888,0.0,0.0,5062207.888,105266.000,0.000,-4252.000,5163221.888
11188,30/06/2018,ITUB3,4.771360e+06,0.0,4.771360e+06,-4.423934e+06,347425.792,0.0,3373552.928,-142199.000,3578779.720,0,3578779.720,0.0,0.0,3578779.720,690689.024,0.000,-2657.000,4266811.744
11189,31/03/2018,ITUB3,1.080704e+06,0.0,1.080704e+06,-7.041750e+05,376529.024,0.0,4255811.072,-158635.000,4473705.096,0,4473705.096,0.0,0.0,4473705.096,903884.992,0.000,1796.000,5379386.088
11190,31/12/2017,ITUB3,2.375363e+06,0.0,2.375363e+06,-1.427571e+06,947792.128,0.0,3645013.904,-129016.000,4463790.032,0,4463790.032,0.0,0.0,4463790.032,-295908.992,201802.000,-13264.000,4356419.040
11191,30/09/2017,ITUB3,6.489080e+05,0.0,6.489080e+05,-5.687600e+04,592032.032,0.0,6414998.136,-242747.000,6764283.168,0,6764283.168,0.0,0.0,6764283.168,-1133926.016,0.000,-3103.000,5627254.152
11192,30/06/2017,ITUB3,1.737642e+06,0.0,1.737642e+06,-1.055420e+06,682221.952,0.0,3632312.056,-141068.000,4173466.008,0,4173466.008,0.0,0.0,4173466.008,644646.016,0.000,-1353.000,4816759.024
11193,31/03/2017,ITUB3,7.032170e+05,0.0,7.032170e+05,-1.370810e+05,566136.032,0.0,4406376.960,-193469.000,4779043.992,0,4779043.992,0.0,0.0,4779043.992,814510.976,0.000,-16547.000,5577007.968
